# Лекция DVP 01. Введение в Feature Engineering

**Feature Engineering** - Генерация признаков.  
**Признак (фича, feature)** - это переменная, которая описывает отдельную характеристику объекта.

*Предварительно была вводная лекция по основам*

## Практическая часть

In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

В качестве примера не будем брать никаких датасетов, а создадим его вручную.  
Создаем семпл из 500 значений. Зависимость стоимости от возраста владельца, длины и ширины помещения)

In [9]:
# Создаём сэмпл из 500 значений
n_samples = 500
# Задаем, случайным образом параметры возраста, длины и ширины помещений. Это признаки
age_owner = np.random.choice(90, n_samples) + 21
length = np.random.choice(120, n_samples) + 15
width = np.random.choice(80, n_samples) + 10
# Целевая метрика: стоимость помещения
price = length * width * 100
# Заносим это все в датафрейм
data = pd.DataFrame({'age_owner': age_owner, 'length': length, 'width': width, 'price': price})
data.head(5)

,age_owner,length,width,price
0,64,46,73,335800
1,72,108,35,378000
2,95,86,82,705200
3,84,92,42,386400
4,73,44,82,360800


Задаем функцию и линейную регрессионную модель, получаем прогноз и рассчитываем ср. абс. ошибку

In [18]:
# Ответ на вопрос "Каким образом мы определим, что какая-то метрика не нужна"
# Для этого рассчитаем среднюю абсолютную ошибку. Исспользуем mean_absolute_error из sklearn
from sklearn.metrics import mean_absolute_error

# Задаем X и Y
X = data[['age_owner', 'length', 'width']]
y = data['price']

# Создаем линейную модель для данной функции
reg = LinearRegression().fit(X, y) # .fit(X, y) - обучает линейную регрессию

# Печать результата
print('Weights: {}'.format(reg.coef_)) # Веса
print('Bias: {}'.format(reg.intercept_)) # Сдвиг(подгонка модели)

# Предсказываем результат и рассчитываем ошибку
pred_values = reg.predict(data[['age_owner', 'length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 221.24848569 4834.01232071 7425.08231347]
Bias: -377941.372488589
Error: 60161.738851695314


**Результат действия модели**:  
Стоимость =  221 х Возраст  +  4834 х Длина + 7444 х Ширина
Ошибка = 60 161 рублей

In [14]:
# Оценим масштаб ошибки:
y

0      335800
1      378000
2      705200
3      386400
4      360800
        ...  
495    275200
496    146300
497    401500
498    518400
499    318200
Name: price, Length: 500, dtype: int32

In [16]:
# Рассчитаем разницу:
y - pred_values
# Разница огромная

0      -64814.106231
1      -41939.730089
2       37540.957063
3      -10826.090980
4      -98963.058782
           ...      
495     -7930.018123
496   -206995.626185
497    -49036.423203
498     19624.924593
499      5649.763650
Name: price, Length: 500, dtype: float64

**Причина ошибки**:   
Модель рассчитывались исходя из параметров возраст, ширина и длина, т.е. на вход использовались некорректные данные

Изменим модель, будем учитывать только длину и ширину

In [17]:
X = data[['length', 'width']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4826.72593105 7428.10211253]
Bias: -362613.49259118503
Error: 60291.38393370125


**Результат действия модели**:  
Стоимость =  4826 х Длина + 7428 х Ширина
Ошибка = 60 291 рублей  
Таким образом, ошибка практически не изменилась

Сумма длины и ширины не работает. Создадим новый признак: площадь квартиры

In [21]:
# Создаем новый признак
data['mult'] = data['length'] * data['width']
data.head(5)

,age_owner,length,width,price,mult
0,64,46,73,335800,3358
1,72,108,35,378000,3780
2,95,86,82,705200,7052
3,84,92,42,386400,3864
4,73,44,82,360800,3608


Создадим модель 3. На X только площадь помещения, а на Y стоимость

In [20]:
X = data[['mult']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [100.]
Bias: -5.820766091346741e-11
Error: 1.775333657860756e-11


**Результат действия модели**:  
Стоимость =  100 х Площадь
Ошибка ~ 0 рублей  
Таким образом, ошибка близка к нулю. Т.е. стоимость зависит от площади, что очевидно.